<a href="https://colab.research.google.com/github/KrisNguyen135/Project-Euler/blob/master/written_solutions/p630.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project Euler 630: Crossed lines

Link to the problem prompt: [https://projecteuler.net/problem=630](https://projecteuler.net/problem=630)

The first $n$ points can be generated in a naive manner with $O(n)$ time complexity. This is done using the `generate_points()` function below.

For each pair of points $(x_A, y_A)$ and $(x_B, y_B)$, the line that goes through them is either (1) $x = x_A$ if $x_A = x_B$ or (2) $y = \frac{y_A - y_B}{x_A - x_B} x + \frac{x_A y_B - x_B y_A}{x_A - x_B}$ otherwise. Iterating through all the pairs of points $(T_{2k - 1}, T_{2k})$, we only need to keep track of the unique lines, which corresponds to the unique tuples $(m = \frac{y_A - y_B}{x_A - x_B}, c = \frac{x_A y_B - x_B y_A}{x_A - x_B})$. This is done using the `get_m_and_c()` function below.

In [0]:
from collections import Counter
import numpy as np

from tqdm import tqdm

In [0]:
MOD1 = 50515093
MOD2 = 2000


def generate_points(n):
    points = []
    temp_s = 290797

    # while len(points) < n:
    for _ in tqdm(range(n)):
        temp_s = pow(temp_s, 2, MOD1)
        x = (temp_s % MOD2) - 1000

        temp_s = pow(temp_s, 2, MOD1)
        y = (temp_s % MOD2) - 1000

        points.append((x, y))

    return points


def get_m_and_c(points):
    m_and_cs = set()

    for id1 in tqdm(range(len(points) - 1)):
        for id2 in range(id1 + 1, len(points)):
            xa, ya = points[id1]
            xb, yb = points[id2]

            if xa == xb:
                m_and_cs.add((None, xa))
            else:
                m = (ya - yb) / (xa - xb)
                c = (xa * yb - xb * ya) / (xa - xb)

                m_and_cs.add((m, c))

    return m_and_cs

In [39]:
points = generate_points(2500)


100%|██████████| 2500/2500 [00:00<00:00, 154236.38it/s]


In [0]:
m_and_cs = get_m_and_c(points)

Finally, if two unique lines share the same slope $m$, they are parallel to each other. We group the unique lines computed above so that lines in the same group are parallel to one another and record the number of items in each group. This is done using the `count_parallel_lines()` function.

Consider the array of item counts among the groups $(c_1, c_2, ..., c_k)$. We need to compute 2 times the number of intersections among the lines the generated these counts. Between a group with $c_i$ parallel lines and other group with $c_j$ parallel lines, there are $c_i \times c_j$ intersections (an intuitive way to think about this is a grid made up from the two groups of parallel lines). So the final answer is 2 times the following sum of pairwise products among the count array $(c_1, c_2, ..., c_k)$:

$$2 \sum_{i \neq j} c_i \times c_j$$

A brute-force approach has an $O(n^2)$ time complexity, with $n$ being the number of distinct groups. This is infeasible when the number of points at the beginning is 2500. Instead, we can rewrite the above expression as:

$$\left( \sum_i c_i \right)^2 - \sum_i c_i^2$$

These two expressions can be computed efficiently using NumPy:

In [0]:
ms = [item[0] for item in m_and_cs]
line_counter = Counter(ms)

In [42]:
counts = np.array([item[1] for item in line_counter.items()])

np.sum(counts) ** 2 - np.sum(counts ** 2)

9669182880384